In [ ]:
import ollama
import json
from faker import Faker
from datetime import datetime, timedelta
import random

class CustomerSupportChatbot:
    def __init__(self):
        self.business_data = self.load_business_data()
        self.messages = []
        self.model = self.select_model()

    def load_business_data(self):
        fake = Faker()
        
        # Generate products
        products = []
        categories = ["Electronics", "Home & Garden", "Fashion", "Books", "Sports & Outdoors"]
        for _ in range(20):
            category = random.choice(categories)
            product_name = f"{fake.word().capitalize()} {fake.random_element(['Pro', 'Ultra', 'Lite', 'Max', 'Elite'])}"
            products.append({
                "name": product_name,
                "category": category,
                "price": round(random.uniform(9.99, 199.99), 2),
                "description": fake.sentence(),
                "sku": fake.ean(length=8),
                "in_stock": random.choice([True, False])
            })
        
        # Generate FAQs
        faqs = [
            {"question": "What are your business hours?", "answer": "We're open Monday to Friday, 9 AM to 6 PM, and Saturday 10 AM to 4 PM."},
            {"question": "Do you offer refunds?", "answer": "Yes, we offer refunds within 30 days of purchase for unused items in original condition."},
            {"question": "How can I track my order?", "answer": "You can track your order by logging into your account or using the tracking number sent to your email."},
            {"question": "Do you ship internationally?", "answer": "Yes, we ship to most countries. International shipping rates apply."},
            {"question": "What payment methods do you accept?", "answer": "We accept all major credit cards, PayPal, and Apple Pay."},
            {"question": "How long does shipping usually take?", "answer": "Domestic shipping typically takes 3-5 business days. International shipping can take 7-14 business days."},
            {"question": "Do you offer gift wrapping?", "answer": "Yes, we offer gift wrapping for a small additional fee. You can select this option at checkout."},
            {"question": "What is your return policy?", "answer": "You can return items within 30 days of purchase. Items must be unused and in original packaging."},
            {"question": "Do you have a physical store?", "answer": "Yes, we have a flagship store located at 123 Main St, Anytown, USA. You can also shop online."},
            {"question": "How can I contact customer support?", "answer": "You can reach our customer support team via email at support@ourbusiness.com or by phone at 1-800-123-4567."}
        ]
        
        # Generate booking slots for the next 7 days
        booking_slots = []
        for i in range(7):
            date = datetime.now() + timedelta(days=i)
            for hour in range(9, 18):  # 9 AM to 5 PM
                if random.random() > 0.3:  # 70% chance of slot being available
                    slot = date.replace(hour=hour, minute=0, second=0, microsecond=0)
                    booking_slots.append(slot.strftime("%Y-%m-%d %I:%M %p"))
        
        # Generate services
        services = [
            {"name": "Basic Consultation", "duration": "30 minutes", "price": 50},
            {"name": "Advanced Troubleshooting", "duration": "1 hour", "price": 100},
            {"name": "Custom Solution Design", "duration": "2 hours", "price": 200},
            {"name": "Emergency Support", "duration": "Varies", "price": "150/hour"},
            {"name": "Software Installation", "duration": "1 hour", "price": 75},
        ]
        
        return {
            "products": products,
            "faqs": faqs,
            "booking_slots": booking_slots,
            "services": services
        }

    def select_model(self):
        models = [model["name"] for model in ollama.list()["models"] if "llama" in model["name"].lower()]
        print("Available models:")
        for i, model in enumerate(models):
            print(f"{i+1}. {model}")
        selection = int(input("Choose your LLaMA model (enter the number): ")) - 1
        return models[selection]

    def generate_response(self, prompt):
        system_message = f"""You are an AI assistant for a small business. Use the following information to assist customers:
        Products: {json.dumps(self.business_data['products'])}
        FAQs: {json.dumps(self.business_data['faqs'])}
        Available Booking Slots: {json.dumps(self.business_data['booking_slots'])}
        Services: {json.dumps(self.business_data['services'])}

        Provide helpful, concise responses. For bookings, confirm available slots. For product and service inquiries, give accurate information."""
        
        messages = [
            {"role": "system", "content": system_message},
            *self.messages,
            {"role": "user", "content": prompt}
        ]

        response = ollama.chat(
            model=self.model,
            messages=messages,
        )
        return response['message']['content']

    def chat(self):
        print("Welcome to our AI-powered customer support!")
        print("We can help you with:")
        print("- Product information")
        print("- Booking appointments")
        print("- Answering common questions")
        print("- And more!")
        print("\nType 'quit' to end the conversation.\n")

        while True:
            user_input = input("You: ")
            if user_input.lower() == 'quit':
                print("Thank you for using our customer support. Goodbye!")
                break

            self.messages.append({"role": "user", "content": user_input})
            response = self.generate_response(user_input)
            print(f"AI: {response}")
            self.messages.append({"role": "assistant", "content": response})

if __name__ == "__main__":
    chatbot = CustomerSupportChatbot()
    chatbot.chat()